In [ ]:
!pip install numpy==1.26.4 scipy==1.11.4 scikit-surprise==1.1.4 --upgrade --no-cache-dir


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 222.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:
      Successfully uninstalled scipy-1.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.11.4 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompat

1. Доцільна реалізація

In [ ]:
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
data = Dataset.load_builtin('ml-100k')

print("Перші 5 рядків датасету:")
for row in data.raw_ratings[:5]:
    print(row)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 Y


Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
Перші 5 рядків датасету:
('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


In [ ]:
data = Dataset.load_builtin('ml-100k')

print("Перші 5 рядків датасету з поясненням:")
for row in data.raw_ratings[:5]:
    user_id, movie_id, rating, timestamp = row
    print(f"Користувач {user_id} оцінив фільм {movie_id} на {rating} балів (часова позначка: {timestamp})")


Перші 5 рядків датасету з поясненням:
Користувач 196 оцінив фільм 242 на 3.0 балів (часова позначка: 881250949)
Користувач 186 оцінив фільм 302 на 3.0 балів (часова позначка: 891717742)
Користувач 22 оцінив фільм 377 на 1.0 балів (часова позначка: 878887116)
Користувач 244 оцінив фільм 51 на 2.0 балів (часова позначка: 880606923)
Користувач 166 оцінив фільм 346 на 1.0 балів (часова позначка: 886397596)


In [ ]:
trainset, testset = train_test_split(data, test_size=0.2)

# Параметри для підбору через GridSearchCV для SVD
param_grid_svd = {
    'n_factors': [50, 100],
    'lr_all': [0.005, 0.010],
    'reg_all': [0.02, 0.05]
}

# Параметри для підбору через GridSearchCV для KNNBasic
param_grid_knn = {
    'k': [20, 40],
    'min_k': [1, 5],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [False, True]
    }
}

grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=3)
grid_search_svd.fit(data)

grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=3)
grid_search_knn.fit(data)



Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Comput

In [ ]:
best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

# Вибір моделі з найменшим MAE
if grid_search_svd.best_score['mae'] < grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")



Найкращий MAE для SVD: 0.733259675415046
Найкращий MAE для KNNBasic: 0.8083344634757443
Вибраний алгоритм: SVD


In [ ]:

best_model.fit(trainset)

# Виведення рекомендацій для конкретного користувача
user_id = str(196)  # ID користувача для якого ми хочемо отримати рекомендації
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

# Отримання рекомендацій
all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

# Прогнозування рейтингів для нерецензованих фільмів
predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

# Виведення 10 найкращих фільмів
print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Кількість рецензій користувача 196: 31
Топ-10 фільмів, рекомендованих для користувача:
Фільм 64 з прогнозованим рейтингом 4.74
Фільм 313 з прогнозованим рейтингом 4.65
Фільм 318 з прогнозованим рейтингом 4.58
Фільм 515 з прогнозованим рейтингом 4.47
Фільм 169 з прогнозованим рейтингом 4.46
Фільм 114 з прогнозованим рейтингом 4.45
Фільм 98 з прогнозованим рейтингом 4.39
Фільм 963 з прогнозованим рейтингом 4.37
Фільм 357 з прогнозованим рейтингом 4.36
Фільм 651 з прогнозованим рейтингом 4.35
